In [ ]:
import pandas as pd

In [ ]:
oe = pd.read_csv('./Raw files/gnomad_v4_constraint.tsv', sep='\t')
oe = oe.sort_values(['gene','mane_select'], ascending=[True,False])
oe = oe[['gene','lof.oe','lof.oe_ci.upper']].drop_duplicates('gene').rename({'lof.oe':'oe','lof.oe_ci.upper':'loeuf'},axis=1)
cv = pd.read_pickle('./Cleaned files/cv.pkl')
cv_dir = pd.read_pickle('./Cleaned files/cv_dir.pkl')
l2g = pd.read_pickle('./Cleaned files/l2g.pkl')
l2g_dir = pd.read_pickle('./Cleaned files/l2g_dir.pkl')
apd = pd.read_pickle('./Cleaned files/apd.pkl')
apdz = pd.read_pickle('./Cleaned files/apdz.pkl')
apdz_dir = pd.read_pickle('./Cleaned files/apdz_dir.pkl')

phecodes = pd.read_excel('./Phecode/phecode_counts_v2.xlsx')
phecodes = phecodes.loc[phecodes['include'] == 'Y']['phecode'].unique()

ot = pd.read_pickle('./Cleaned files/opentargets.pkl').sort_values(['id','phecode','gene','phase'], ascending=False).drop_duplicates(['id','phecode','gene'])
ot.loc[ot['phase'] > 0, 'indication'] = 1
ot = ot.sort_values(['id','gene','moa']).drop_duplicates()
ot.loc[ot['moa'] == 'inhibitor', 'inhibitor'] = 1
ot.loc[ot['moa'] == 'activator', 'activator'] = 1
ot = ot.drop('moa',axis=1)
print('OT all', ot['id'].nunique(), ot['gene'].nunique(), ot['indication'].sum())
temp = ot.loc[ot['phecode'].isin(phecodes)]
print('OT phecodes', temp['id'].nunique(), temp['gene'].nunique(), temp['indication'].sum())

sd = pd.read_pickle('./Cleaned files/sider.pkl').sort_values(['id','phecode','gene'], ascending=False).drop_duplicates(['id','phecode','gene'])
sd.loc[sd['phecode'].notna(), 'indication'] = 1
sd.loc[sd['phecode'].notna(), 'phase'] = 4
sd.loc[sd['moa'] == 'inhibitor', 'inhibitor'] = 1
sd.loc[sd['moa'] == 'activator', 'activator'] = 1
sd = sd.drop('moa',axis=1)
print('SD all', sd['id'].nunique(), sd['gene'].nunique(), sd['indication'].sum())
temp = sd.loc[sd['phecode'].isin(phecodes)]
print('SD phecodes', temp['id'].nunique(), temp['gene'].nunique(), temp['indication'].sum())

otu = ot[['gene']].drop_duplicates()
otp = list(ot['phecode'].dropna().unique())
otu = pd.concat([otu.assign(phecode=value) for value in otp], ignore_index=True)
otu = otu.loc[otu['phecode'].isin(phecodes)]
otu = otu.merge(ot.loc[ot['indication'] == 1].groupby(['gene','phecode'])['id'].nunique().reset_index().rename({'id':'n_drugs'},axis=1), how='left')
otu = otu.merge(ot.sort_values('phase', ascending=False).drop_duplicates(['gene','phecode'])[['gene','phecode','indication','phase','inhibitor','activator']], how='left')
otu.loc[otu['phase'] == 0.5, 'phase'] = 1
otu.loc[otu['phase'] == -1, 'phase'] = 0

sdu = sd[['gene']].drop_duplicates()
sdp = list(sd['phecode'].dropna().unique())
sdu = pd.concat([sdu.assign(phecode=value) for value in sdp], ignore_index=True)
sdu = sdu.loc[sdu['phecode'].isin(phecodes)]
sdu = sdu.merge(sd.loc[sd['indication'] == 1].groupby(['gene','phecode'])['id'].nunique().reset_index().rename({'id':'n_drugs'},axis=1), how='left')
sdu = sdu.merge(sd.sort_values('phase', ascending=False).drop_duplicates(['gene','phecode'])[['gene','phecode','indication','phase','inhibitor','activator']], how='left')

print(otu['phecode'].nunique())
print(sdu['phecode'].nunique())


OT all 4987 1554 46176.0
OT phecodes 1286 961 8870.0
SD all 908 823 13601.0
SD phecodes 424 548 3752.0
78
79


In [ ]:
gc = cv.merge(cv_dir, on=['phecode','gene'], how='outer').merge(l2g, on=['phecode','gene'], how='outer')\
        .merge(l2g_dir, on=['phecode','gene'], how='outer').merge(apd, on=['phecode','gene'], how='outer')\
        .merge(apdz, on=['phecode','gene'], how='outer').merge(apdz_dir, on=['phecode','gene'], how='outer').drop_duplicates()

otu = otu.merge(gc, on=['phecode','gene'], how='left').merge(oe[['gene','oe']], how='left')
otu.loc[otu['oe'].isna(), 'oe'] = otu['oe'].median()
otu = otu.join(pd.get_dummies(otu['phecode'].str[:2]).astype(int)).drop_duplicates()
otu = otu.fillna(0)
otu.loc[(otu['p_g'] == 0) & (otu['c_g'] == 1), 'c_only_g'] = 1
otu.loc[(otu['p_e'] == 0) & (otu['c_e'] == 1), 'c_only_e'] = 1
otu.loc[(otu['p_b'] == 0) & (otu['c_b'] == 1), 'c_only_b'] = 1
otu.loc[(otu['p_g'] == 0) & (otu['b_g'] == 1), 'b_only_g'] = 1
otu.loc[(otu['p_e'] == 0) & (otu['b_e'] == 1), 'b_only_e'] = 1
otu.loc[(otu['p_b'] == 0) & (otu['b_b'] == 1), 'b_only_b'] = 1
otu = otu.fillna(0)
otu.to_csv('./Datasets/ot_input.csv', index=None)

sdu = sdu.merge(gc, on=['phecode','gene'], how='left').merge(oe[['gene','oe']], how='left')
sdu.loc[sdu['oe'].isna(), 'oe'] = sdu['oe'].median()
sdu = sdu.join(pd.get_dummies(sdu['phecode'].str[:2]).astype(int)).drop_duplicates()
sdu = sdu.fillna(0)
sdu.loc[(sdu['p_g'] == 0) & (sdu['c_g'] == 1), 'c_only_g'] = 1
sdu.loc[(sdu['p_e'] == 0) & (sdu['c_e'] == 1), 'c_only_e'] = 1
sdu.loc[(sdu['p_b'] == 0) & (sdu['c_b'] == 1), 'c_only_b'] = 1
sdu.loc[(sdu['p_g'] == 0) & (sdu['b_g'] == 1), 'b_only_g'] = 1
sdu.loc[(sdu['p_e'] == 0) & (sdu['b_e'] == 1), 'b_only_e'] = 1
sdu.loc[(sdu['p_b'] == 0) & (sdu['b_b'] == 1), 'b_only_b'] = 1
sdu = sdu.fillna(0)
sdu.to_csv('./Datasets/sider_input.csv', index=None)

all_gp = gc.drop_duplicates(['gene','phecode']).fillna(0)
all_gp = all_gp.loc[all_gp['phecode'].isin(phecodes)]
all_gp.to_csv('./Datasets/all_gp.csv', index=False)